# Preprocess Bureau of Labor Statistic Data

## Occupational Requirement Survey

From `or.txt` (readme):
```
The series_id (ORUP1000066700000560) can be broken out into:

Code                                    Value(Example)

Survey abbreviation             =               OR
Seasonal(code)                  =               U
Requirement_code                =               P
Ownership_code                  =               1
Industry_code                   =               0000
Occupation_code                 =               667
Job_characteristic_code         =               000
Estimate_code                   =               00560
```

Goal: Extract a "job id" to `estimate_code` from each `series_id`.

Load all data (`or.data.1.AllData`) into a pandas DataFrame

In [1]:
import matplotlib

In [2]:
import pandas

series = pandas.read_csv('ordata/or.data.1.AllData', delimiter='\t')
series['value'] = pandas.to_numeric(series['value'], errors='coerce')

In [3]:
series.head()

,series_id,year,period,value,footnote_codes
0,ORUC1000000000000728,2017,A01,5.1,6
1,ORUC1000000000001030,2017,A01,29.0,
2,ORUC1000000000001031,2017,A01,48.2,7
3,ORUC1000000000001032,2017,A01,15.6,
4,ORUC1000000000001033,2017,A01,5.8,


Check if `occupation_code` is unique (viable candidate for a "job id"

In [4]:
occupation_code = series['series_id'].map(lambda a: a[9:12])
occupation_code.head()

0    000
1    000
2    000
3    000
4    000
Name: series_id, dtype: object

In [5]:
series_prefix = series['series_id'].map(lambda a: a[:3] + a[4:15])
series_prefix.head()

0    ORU10000000000
1    ORU10000000000
2    ORU10000000000
3    ORU10000000000
4    ORU10000000000
Name: series_id, dtype: object

In [6]:
len(series_prefix.unique()), len(occupation_code.unique())

(338, 338)

`occupation_code` is a viable "job id" key (only one row for each metric for each occupation code/job). Map to `soc_code` (really O\*NET-SOC 2010 code) and convert to proper `soc_code` (drop the last two digits) for better combination with other datasets from BLS.

In [7]:
series['occupation_code'] = series['series_id'].map(lambda a: a[9:12])

In [8]:
occupations = pandas.read_csv(
    'ordata/or.occupation', 
    delimiter='\t', 
    index_col=False, 
    dtype={'occupation_code': str, 'soc_code': str}, 
    usecols=['occupation_code', 'soc_code', 'display_level', 'occupation_text']
)
occupations['soc_code'] = occupations['soc_code'].map(lambda a: a[:6]) # convert from ONETSOC to SOC code
occupations['soc_cat'] = occupations['soc_code'].map(lambda a: a[:2])
occupations['is_category'] = occupations['display_level'] == 0
del occupations['display_level']
occupations.head()

,occupation_code,soc_code,occupation_text,soc_cat,is_category
0,000,000000,All Workers,00,True
1,001,111011,Chief Executives,11,False
2,003,111021,General and Operations Managers,11,False
3,007,112021,Marketing Managers,11,False
4,008,112022,Sales Managers,11,False


In [9]:
series = occupations.merge(series, on='occupation_code')
series.head()

,occupation_code,soc_code,occupation_text,soc_cat,is_category,series_id,year,period,value,footnote_codes
0,000,000000,All Workers,00,True,ORUC1000000000000728,2017,A01,5.1,6
1,000,000000,All Workers,00,True,ORUC1000000000001030,2017,A01,29.0,
2,000,000000,All Workers,00,True,ORUC1000000000001031,2017,A01,48.2,7
3,000,000000,All Workers,00,True,ORUC1000000000001032,2017,A01,15.6,
4,000,000000,All Workers,00,True,ORUC1000000000001033,2017,A01,5.8,


In [10]:
series = series[series['is_category'] != True]
del series['is_category']
series.head()

,occupation_code,soc_code,occupation_text,soc_cat,series_id,year,period,value,footnote_codes
272,001,111011,Chief Executives,11,ORUC1000000100001033,2017,A01,31.4,
273,001,111011,Chief Executives,11,ORUC1000000100001034,2017,A01,40.4,7
274,001,111011,Chief Executives,11,ORUC1000000100001039,2017,A01,30.1,7
275,001,111011,Chief Executives,11,ORUC1000000100001040,2017,A01,24.2,
276,001,111011,Chief Executives,11,ORUC1000000100001047,2017,A01,32.0,


Extract `estimate_code` aka metric id

In [11]:
series['estimate_code'] = series['series_id'].map(lambda a: a[15:20])
series.head()

,occupation_code,soc_code,occupation_text,soc_cat,series_id,year,period,value,footnote_codes,estimate_code
272,001,111011,Chief Executives,11,ORUC1000000100001033,2017,A01,31.4,,01033
273,001,111011,Chief Executives,11,ORUC1000000100001034,2017,A01,40.4,7,01034
274,001,111011,Chief Executives,11,ORUC1000000100001039,2017,A01,30.1,7,01039
275,001,111011,Chief Executives,11,ORUC1000000100001040,2017,A01,24.2,,01040
276,001,111011,Chief Executives,11,ORUC1000000100001047,2017,A01,32.0,,01047


In [12]:
len(series['estimate_code'].unique())

329

Rows can be unique identified by (`occupation_code` and `estimate_code`) 

In [13]:
# sanity check
num_occs = len(series['occupation_code'].unique())
num_ests = len(series['estimate_code'].unique())
print('{} * {} = {}'.format(num_occs, num_ests, num_occs * num_ests))
print(len(series))
# because some of the data is 0/missing

315 * 329 = 103635
35376


Merge data (average) by (`soc_code` and `estimate_code`).
Then, convert the data from rows with keyed by (`occupation_code`, `estimate_code`) to a matrix of with `occupation_code` rows and `estimate_code` columns.

In [14]:
estimate_values_by_soc = series.groupby(by=('soc_code', 'estimate_code'))['value'].mean()
orsdata = estimate_values_by_soc.to_frame().pivot_table(
    index="soc_code", 
    columns="estimate_code", 
    values="value"
)
orsdata.head()

estimate_code,00064,00065,00066,00067,00068,00069,00070,00071,00072,00075,...,01076,01077,01080,01081,01084,01085,01087,01088,01090,01091
soc_code,,,,,,,,,,,,,,,,,,,,,
111011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.7,56.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111021,NaN,NaN,NaN,NaN,NaN,6.1,16.7,51.3,20.9,NaN,...,11.0,19.1,70.4,29.6,10.7,8.3,NaN,15.7,NaN,18.4
112021,NaN,NaN,NaN,NaN,NaN,NaN,14.6,57.6,24.9,NaN,...,NaN,21.8,65.3,34.7,NaN,NaN,NaN,NaN,NaN,NaN
112022,NaN,NaN,NaN,NaN,NaN,NaN,19.8,56.2,11.6,NaN,...,NaN,31.7,83.7,16.3,NaN,32.7,NaN,NaN,NaN,33.4
112031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
orsdata.to_csv('ordata-processed.csv')

In [16]:
categories = pandas.DataFrame(occupations[occupations['is_category'] == True], columns=('soc_cat', 'occupation_text'))
categories.columns = ['soc_cat', 'category_name']
categories.head()

,soc_cat,category_name
0,00,All Workers
65,11,Management Occupations
66,13,Business and Financial Operations Occupations
69,15,Computer and Mathematical Occupations
73,17,Architecture and Engineering Occupations


In [17]:
occupations = occupations.merge(categories, on='soc_cat')
occupations.set_index('soc_code', inplace=True)
occupations.head()

,occupation_code,occupation_text,soc_cat,is_category,category_name
soc_code,,,,,
000000,000,All Workers,00,True,All Workers
111011,001,Chief Executives,11,False,Management Occupations
111021,003,General and Operations Managers,11,False,Management Occupations
112021,007,Marketing Managers,11,False,Management Occupations
112022,008,Sales Managers,11,False,Management Occupations


In [90]:
category_map = pandas.DataFrame({'soc_code': occupations.index.unique()})
print(len(category_map))
category_map.set_index('soc_code', inplace=True)
category_map = category_map.join(occupations, how='inner')
print(len(category_map))
category_map.head()

320
338


,occupation_code,occupation_text,soc_cat,is_category,category_name
soc_code,,,,,
000000,000,All Workers,00,True,All Workers
110000,27P,Management Occupations,11,True,Management Occupations
111011,001,Chief Executives,11,False,Management Occupations
111021,003,General and Operations Managers,11,False,Management Occupations
112021,007,Marketing Managers,11,False,Management Occupations


In [99]:
category_map = pandas.DataFrame(occupations['category_name'])
category_map = category_map[~category_map.index.duplicated(keep='first')]
category_map.head()

,category_name
soc_code,
000000,All Workers
111011,Management Occupations
111021,Management Occupations
112021,Management Occupations
112022,Management Occupations


## Occupational Employment Survey

Add several metrics from OES as columns (national and state-level). TODO does this format work for state level data in the cloropath?

From `oe.txt`:
```
The series_id (OEUM000040000000000000001) can be broken out into:

Code					Value(Example)

survey abbreviation =       OE
seasonal(code)      =       U
areatype-code       =       M
area_code           =       0000400
industry_code       =       000000
occupation_code     =       000000 
datatype_code       =       01
```

First, load data and extract `occupation_code` (SOC format), `area_code`, `areatype-code`, and `datatype_code`.

In [19]:
oesdata = pandas.read_csv(
    'oedata/oe.data.1.AllData', 
    delimiter='\t',      
    usecols=['series_id', 'value'])
oesdata['value'] = pandas.to_numeric(oesdata['value'], errors='coerce')
oesdata['soc_code'] = oesdata['series_id'].map(lambda s: s[17:23])
oesdata['area_code'] = oesdata['series_id'].map(lambda s: s[4:11])
oesdata['areatype-code'] = oesdata['series_id'].map(lambda s: s[3:4])
oesdata['datatype_code'] = oesdata['series_id'].map(lambda s: s[23:25])
oesdata.head()

,series_id,value,soc_code,area_code,areatype-code,datatype_code
0,OEUM001018000000000000001,64450.00,000000,0010180,M,01
1,OEUM001018000000000000002,2.30,000000,0010180,M,02
2,OEUM001018000000000000003,19.88,000000,0010180,M,03
3,OEUM001018000000000000004,41350.00,000000,0010180,M,04
4,OEUM001018000000000000005,2.20,000000,0010180,M,05


Filter out municipal level data (national and state only)

In [20]:
len(oesdata)

6253097

In [21]:
oesdata = oesdata[oesdata['areatype-code'] == 'N']
# oesdata = oesdata[oesdata['areatype-code'] != 'M']
len(oesdata)

1954290

Add in area names

In [22]:
areas = pandas.read_csv(
    'oedata/oe.area', 
    delimiter='\t',
    usecols=['area_code', 'area_name'],
    converters={'area_code': str},
    index_col=False
)
areas.head()

,area_code,area_name
0,0000000,National
1,0011500,"Anniston-Oxford-Jacksonville, AL"
2,0012220,"Auburn-Opelika, AL"
3,0013820,"Birmingham-Hoover, AL"
4,0019300,"Daphne-Fairhope-Foley, AL"


In [23]:
oesdata = oesdata.merge(areas, on='area_code')
oesdata.head()

,series_id,value,soc_code,area_code,areatype-code,datatype_code,area_name
0,OEUN000000000000000000001,1.425492e+08,000000,0000000,N,01,National
1,OEUN000000000000000000002,1.000000e-01,000000,0000000,N,02,National
2,OEUN000000000000000000003,2.434000e+01,000000,0000000,N,03,National
3,OEUN000000000000000000004,5.062000e+04,000000,0000000,N,04,National
4,OEUN000000000000000000005,1.000000e-01,000000,0000000,N,05,National


From `oe.datatype`:
```
datatype_code datatype_name
01	Employment	
02	Employment percent relative standard error	
03	Hourly mean wage	
04	Annual mean wage	
05	Wage percent relative standard error	
06	Hourly 10th percentile wage	
07	Hourly 25th percentile wage	
08	Hourly median wage	
09	Hourly 75th percentile wage	
10	Hourly 90th percentile wage	
11	Annual 10th percentile wage	
12	Annual 25th percentile wage	
13	Annual median wage	
14	Annual 75th percentile wage	
15	Annual 90th percentile wage	
16	Employment per 1,000 jobs	
17	Location Quotient	
```

In [24]:
oesdata = oesdata[oesdata['datatype_code'].isin(['01', '04', '13',])]
len(oesdata)

390858

In [25]:
oesdata_by_soc_area = oesdata.pivot_table(
#     index=('soc_code', 'area_name'),
    index='soc_code',
    columns='datatype_code', 
    values='value'
)
oesdata_by_soc_area.columns = ['num_employed', 'mean_annual_wage', 'med_annual_wage']
oesdata_by_soc_area.head()

,num_employed,mean_annual_wage,med_annual_wage
soc_code,,,
000000,1.755982e+06,52457.888889,43163.622222
110000,8.819913e+04,116079.287305,100105.567929
111000,2.908770e+04,127168.374165,106168.819599
111011,2.891013e+03,200269.845361,169287.192308
111021,2.578815e+04,120918.173719,101717.995546


In [26]:
# oesdata_by_soc = oesdata_by_soc_area.pivot_table(
#     index='soc_code', 
#     columns='area_name',
#     values=('num_employed', 'med_annual_wage')
# )
# oesdata_by_soc.dropna(how='all', inplace=True)
# oesdata_by_soc.shape

In [27]:
oesdata_by_soc_area.to_csv('oedata-processed.csv')

## Join Calculated ORS metrics and OES data

See `transform_or.ipynb`

In [100]:
calculated_ors = pandas.read_csv('calculated_metrics.csv', converters={'soc_code': str})
calculated_ors.set_index('soc_code', inplace=True)
calculated_ors.head()

,occupation_name,communication,danger,experience,interaction_complexity,pace_of_work,physicality,uncertain_decisions,variety
soc_code,,,,,,,,,
111011,Chief Executives,0.550012,-0.509746,0.501938,1.164292,-0.114775,-0.724120,2.411379,0.598219
111021,General and Operations Managers,0.771077,-0.309912,0.519413,1.883329,0.004275,-0.400566,1.888648,1.744551
112021,Marketing Managers,-0.029500,-0.558746,0.427417,2.550999,-0.080702,-0.883368,2.108915,1.345437
112022,Sales Managers,1.440692,-0.434556,0.484537,3.354653,0.213450,-0.554409,1.848539,2.052344
112031,Public Relations and Fundraising Managers,0.626005,-0.518194,-0.454664,-0.689362,0.063105,-0.358270,-0.964388,-0.895883


In [101]:
len(category_map), len(category_map.index.unique())

(320, 320)

In [102]:
data = calculated_ors.join(category_map, how='inner')
data = data.join(oesdata_by_soc_area, how='inner')
data.head()

,occupation_name,communication,danger,experience,interaction_complexity,pace_of_work,physicality,uncertain_decisions,variety,category_name,num_employed,mean_annual_wage,med_annual_wage
soc_code,,,,,,,,,,,,,
111011,Chief Executives,0.550012,-0.509746,0.501938,1.164292,-0.114775,-0.724120,2.411379,0.598219,Management Occupations,2891.012658,200269.845361,169287.192308
111021,General and Operations Managers,0.771077,-0.309912,0.519413,1.883329,0.004275,-0.400566,1.888648,1.744551,Management Occupations,25788.151448,120918.173719,101717.995546
112021,Marketing Managers,-0.029500,-0.558746,0.427417,2.550999,-0.080702,-0.883368,2.108915,1.345437,Management Occupations,3166.101695,129396.925287,117437.959770
112022,Sales Managers,1.440692,-0.434556,0.484537,3.354653,0.213450,-0.554409,1.848539,2.052344,Management Occupations,5057.162162,127374.364641,112496.805556
112031,Public Relations and Fundraising Managers,0.626005,-0.518194,-0.454664,-0.689362,0.063105,-0.358270,-0.964388,-0.895883,Management Occupations,1832.000000,125821.116505,112109.463415


In [103]:
data.to_csv('job-data.csv')